In [10]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


df = pd.read_csv("flipkart_com-ecommerce_sample.csv", engine="python")
df = df[["product_name", "product_category_tree",]].dropna().reset_index(drop=True)
df = df[:5500]


df['product_category_tree'] = df['product_category_tree'].str.replace('[', '', regex=False).str.replace(']', '', regex=False).str.replace("'", "", regex=False).str.replace(',', ' ', regex=False)
df['product_category_tree'] = df['product_category_tree'].str.lower()

tfidf = TfidfVectorizer(stop_words='english')


tfidf_matrix = tfidf.fit_transform(df['product_category_tree'])


similarity_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)

def recommend_by_cat(cat, num_recommendations=10):

    filtered_df = df[df['product_category_tree'].str.contains(cat.lower(), case=False)]
    
    if filtered_df.empty:
        return f"No product found for category '{cat}'."
    

    recommended_product = []

    for idx in filtered_df.index:

        sim_scores = list(enumerate(similarity_matrix[idx]))
        

        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        

        sim_scores = sim_scores[1:num_recommendations + 1]  
        

        product_indices = [i[0] for i in sim_scores]
        pro = df['product_name'].iloc[product_indices].tolist()
        recommended_product.extend(pro)
    

    recommended_product = list(set(recommended_product))
    
    return recommended_product[:num_recommendations]


recommended_product = recommend_by_cat("footwear", num_recommendations=10)
print("Recommended movies:", recommended_product)


Recommended movies: ['Inc.5 Women Wedges', 'La Briza Ashley Boots', 'Toe Spring Flip Flops', 'iLO Stylish Loafers', "Raymond Men's Printed Formal Blue Shirt", 'Katalogue Women Heels', 'Finesse Loafers', 'Yepme Women Flats', 'Crocs Boys Sandals', 'Port NuWave Football Shoes']
